In [2]:
# %pip install xgboost imblearn

: 

In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.xgboost import XGBclassifier 
from sklearn.metrics import accuracy_score ,confusion_matrix, classification_report 

In [ ]:
df = pd.read_csv('F:\DS\Projects\customer_churn_prediction\data\external\WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head(4)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [16]:
pd.set_option('display.max_columns' ,None)

In [ ]:
df.head(3)

In [19]:
df = df.drop(columns = ['customerID'])

In [ ]:


for col in df.columns:
    print(col , df[col].unique())
    print("---")

In [ ]:
# Filtered the dataFrame 
# where it is Empty
df[df['TotalCharges'] == " "]

len(df[df['TotalCharges'] == " "])
#Rason for empty could be Customer have not used the service 

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace([" "  , "0.0"])

In [33]:
# Converting Monthly charges 
df['TotalCharges']= df['TotalCharges'].astype(float)

In [ ]:
df.info()

In [ ]:
df['Churn'].value_counts()
# Imbalance DataSet

- EDA

In [ ]:
# Separate categorical and numerical features
# categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
# numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [ ]:
df.describe()

In [38]:
def distribution(df, column_name):
    plt.figure(figsize=(5, 3))
    sns.histplot(df[column_name], kde=True)
    plt.title(f"Distribution of {column_name}")

    # Calculate the mean and median values for the column
    col_mean = df[column_name].mean()
    col_median = df[column_name].median()

    # Add vertical lines for mean and median
    plt.axvline(col_mean, color="red", linestyle="--", label="Mean")
    plt.axvline(col_median, color="green", linestyle="-", label="Median")

    plt.legend()
    plt.show()


In [ ]:
distribution(df,'tenure')

In [ ]:
distribution(df,'MonthlyCharges')

In [ ]:
distribution(df,'TotalCharges')

In [ ]:
# 48